In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 16)

# Paths and Names

In [3]:
### paths and names

PROJECT_NAME = 'news-topic-keyphrase-generation-model-dev'
RUN_ID = 'v2'

DATA_PATH = 'data/preprocess_v2.pickle'

MODEL_CHECKPOINT = 'ainize/kobart-news'
model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()

METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, '.log')

run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

print(run_name)
print(output_dir)

!mkdir -p {SAVE_PATH}

ainize_kobart_news_v2
./.log/ainize_kobart_news_v2


In [4]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

env: WANDB_PROJECT=news-topic-keyphrase-generation-model-dev
env: WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find {NOTEBOOK_PATH}.


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Training Args

In [5]:
report_to="wandb"

num_train_epochs = 15
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * NGPU
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=128
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

- 모델과 토크나이저, 그리고 평가지표를 계산하는 데 사용할 함수를 불러옵니다.
- 모델의 config에는 사용하지 않는 설정이 포함되어 있습니다. 삭제합니다.

In [6]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

# Functions

- 모델 훈련을 위해 전처리를 수행해주는 함수를 선언합니다.
- 입력 문장이 input이 되고 요약문이 label이 됩니다.
- tokenizer를 사용해 input_ids로 변환하고 입력값에는 attention_mask를 생성해줍니다.
- 입력 문장에 prefix를 추가하여 성능 실험을 해볼 수 있으나 지금은 적용하지 않겠습니다.

In [8]:
prefix = "generate keyphrases: "

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

- 모델의 예측값과 위에서 전처리한 라벨을 활용하여 평가지표를 출력하는 함수를 선언합니다.
- ROUGE를 평가지표로 사용합니다.
    - ROUGE-N (N-gram) scoring
    - ROUGE-L (Longest Common Subsequence) scoring
        - sentence-level: Compute longest common subsequence (LCS) between two pieces of text. Newlines are ignored. This is called rougeL in this package.
        - summary-level: Newlines in the text are interpreted as sentence boundaries, and the LCS is computed between each pair of reference and candidate sentences, and something called union-LCS is computed. This is called rougeLsum in this package.

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

- csv로 저장된 데이터를 불러와 9:1로 훈련과 검증에 사용합니다.
- 선언한 전처리 함수를 적용하여 데이터세트를 생성합니다.

In [10]:
data_df = pd.read_pickle(DATA_PATH)

In [11]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [12]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map (num_proc=16):   0%|          | 0/1268 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/317 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1268
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 317
})


In [13]:
tokenizer.decode(train_dataset['input_ids'][0])

'<s>generate keyphrases: 공정위, 대기업 기업결합 심사 5년간 918건 \'모두 승인\' 경쟁제한성 판단 4건은 우선승인 뒤 시정조치송재호 "시장독점·경쟁제한 우려...피해자는 국민"송재호 더불어민주당 의원. 2021.9.30/뉴스1 © News1 오대일 기자 서미선 기자 = 공정거래위원회 기업결합심사에서 최근 5년간 대기업집단 기업결합은 단 한 건도 금지된 사례가 없는 것으로 드러났다. 7일 국회 정무위원회 송재호 더불어민주당 의원이 공정위로부터 받은 자료를 분석한 결과, 2017년부터 2021년 6월까지 최근 5년간 자산규모 5조원 이상 대기업집단 기업결합 총 918건은 모두 승인됐다. 금액으로는 145조3000억원 규모다. 공정위는 이 중 4건은 경쟁제한성이 있다고 판단해 시정조치를 내렸다. 우선 기업결합을 승인하되 보완적 조치 이행을 공정위가 관리한다는 것이다. 2018년 CJ헬로비전의 하나방송 인수, 2019년 SK텔레콤의 콘텐츠연합플랫폼 등 인수, 2019년 LG유플러스의 CJ헬로 등 인수, 지난해 SK브로드밴드의 티브로드 등 인수에서 시정조치가 부과됐다. 올해 상반기 기준 대기업집단 기업결합은 196건, 금액은 23조2000억원으로 전체 국내 기업결합의 절반에 가까운 46.4%, 결합금액은 전체의 76.8%를 차지했다. 지난해 같은 기간보다 건수는 87%, 금액은 3배 가까이 증가했다. 송 의원은 "대기업집단 기업결합은 신산업 성장동력 확보에서 긍정적 역할도 있지만, 잠재적 경쟁자를 인수합병해 시장을 독점</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [14]:
tokenizer.decode(train_dataset['labels'][0])

'<s>심사,승인,대기업 기업결합,공정위</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Train

- 상단에서 선언한 HP와 각종 변수들로 training_args를 생성합니다.

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

- 배치 단위로 padding처리를 하여 훈련 속도를 높히기 위해 data_collator를 선언합니다.
- 생성한 객체들을 trainer에 포함시켜 훈련 준비를 마칩니다.

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

- 훈련을 시작합니다.

In [17]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.520800,0.338340,15.044000,1.682400,14.980600,14.997000,28.053600
2,0.343300,0.301407,17.210400,2.731700,16.941300,16.998900,27.589900
3,0.303000,0.297739,19.097600,3.962700,18.926100,18.922500,27.993700
4,0.245000,0.288719,19.242900,3.639000,19.284200,19.312200,27.902200
5,0.224400,0.286380,21.321400,5.576100,21.143800,21.167300,27.164000
6,0.204500,0.289023,19.784400,4.887300,19.622000,19.750800,26.646700
7,0.191700,0.290580,18.111700,4.404400,18.000100,17.881500,25.233400
8,0.172100,0.294094,19.702600,4.415200,19.492000,19.336100,25.217700
9,0.157200,0.298028,18.628200,4.878700,18.270800,18.182200,25.697200
10,0.151900,0.300880,18.992600,4.385700,18.734500,18.544800,25.804400


TrainOutput(global_step=9510, training_loss=0.2072348799677928, metrics={'train_runtime': 2728.9424, 'train_samples_per_second': 6.97, 'train_steps_per_second': 3.485, 'total_flos': 5798593717862400.0, 'train_loss': 0.2072348799677928, 'epoch': 15.0})

- 옵티마이저 스테이트를 포함한 훈련 재개에 사용할 파일들을 삭제해줍니다.
- 추후 불러올 모델과 토크나이저 관련 파일만 남겨둡니다.

In [18]:
# keep = [
#     'added_tokens.json',
#     'config.json',
#     'pytorch_model.bin',
#     'special_tokens_map.json',
#     'tokenizer.json',
#     'tokenizer_config.json',
#     'vocab.txt'
# ]

# ckpts = os.listdir(output_dir)
# for ckpt in ckpts:
#     ckpt = os.path.join(output_dir, ckpt)
#     for item in os.listdir(ckpt):
#         if item not in keep:
#             os.remove(os.path.join(ckpt, item))

# Generate

- 훈련을 마치면 Evaluation Loss 기준 Best 모델이 로드되어 있습니다.
- trainer는 Greedy Search를 수행하도록 설정되어 있습니다.
- Evaluation 데이터를 활용해 모델의 출력을 간단히 살펴봅니다.

In [19]:
preds = trainer.predict(eval_dataset)

In [20]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 체험 마케팅 강화,휴테크,게임,롯데백화점 중동점 브랜드관,안마의자
예측: 휴테크,롯데백화점 중동점 브랜드관,체험 마케팅,위드 코로나

정답: 루카 모드리치,안첼로티,레알마드리드,카세미루
예측: 크카모는 버뮤다 삼각지대

정답: 식중독 주의보,식중독 발생,식중독 환자,식중독 예방
예측: 식중독 주의보,기온,식중독 예방 6대 수칙,식중독,1.6°C,1월,식중독

정답: 황금빛골드바 이벤트,음악 저작권 거래,뮤직카우
예측: 음악카우,황금빛골드바 어텀 이벤트 실시,음악 저작권 거래 플랫폼,연속 이벤트,순금 골드바

정답: 신현빈,재벌집 막내아들,재회,김신록,송중기,순양백화점
예측: 재집 막내아들 송중기,김신록,김신록,김신록,재회

정답: 상장사가 가상자산 발행,발행 규제,자본연 보고서
예측: 상장사의 코인발행,규제,자본시장연구원,상장사,코인발행,자본시장 교란,코인발행,규제

정답: 원불교,나상호 교정원장,교단혁신특별위원회,대각개교절,간담회
예측: 대상호 교정원장,대각개교절,교단 혁신,제107주년 대각개교절,제발절,대신혁명,교단혁신특별위원회

정답: 홈런,알몬테,kbo 리그
예측: K몬테,2021 신한은행 SOL KBO리그,KT 위즈,KT 위즈,KBO리그

정답: 건강 피부,40세 생얼,양미라
예측: 40세 생얼,양미라,젊은 피부,인스타그램

정답: 여야,회동,원내대표,박준영,타협,막판,인준
예측: 박준영 사퇴,인준 대치,여야 원내대표 담판,박준영 해수부 장관 후보자,인사

정답: 노진혁,시작,kbo리그
예측: 노진혁,시작이 좋아,23일 오후,2021 신한은행 SOL KBO리그,NC 다이노스,키움 히어로즈,경기,2회초,천전안타를 치고

정답: 한미,워싱턴,KIDD회의,전작권 논의,한미통합국방협의체 회의,한반도 안보정세 평가,완전한 비핵화
예측: 한,미통합국방협의체 회의,전작권 논의,KIDD회의,한반도 안보정세 평가,한반도의 완전한 비핵화 및 항구적 평화정착,대북정책 공조 방안

정답: 윤순진 위원장,산업통상자원부,국정감사
예측: 윤순진,산업통상자원중소벤처기업위원


예측: 자율주행,화장,나홀로 뒷자리 탑승,테슬라 모델3 운전자

정답: 현세린,OK저축은행 박세리INVITATIONAL
예측: 현세린,비바람,비바람,충주 실크리버 컨트리클럽,OK저축은행,박세리,INVITATIONAL

정답: 반갑지만,'조심',추석 연휴 코로나,가족
예측: 추,연휴,코로나,조심,Noneyphrases

정답: 비주얼,태국재벌,신주아,원피스,포즈
예측: 태국재벌,신주아,운동장,대저택 드레스룸,나홀로 패션쇼

정답: 조선통신사선,선상박물관,문화재청 국립해양문화재연구소,체험형 문화공간,선상박물관 문화기행,역사문화자원,대한민국 4대 관광도시
예측: 선상박물관,해양문화재단,조선통신사선

정답: 이효리,2세 계획,이유,엄마의 희생,노력,사랑
예측: 이리,2세 계획 전한 이유

정답: 천우희,마카롱,형형색색 마카롱,비주얼,비와 당신의 이야기
예측: 천우희,형형색색 마카롱,먹기 아까운 비주얼,인스타그램 스토리,인스타그램 스토리,사진

정답: 연쇄 충돌
예측: 경기차들,중앙선 넘어 연쇄 충돌,5명 다쳐

정답: 윤대통령,'캄보디아 환아',대통령실 초청
예측: 윤대통령 부부,대통령실 초청,캄보디아 환아,대통령실 초청 None

정답: 대장동,유동규,재판,김만배,남욱,정영학,연기
예측: 유장동,김만배·남욱,첫 재판 연기

정답: 삼성 라이온즈,포항 경기,13연패,2022 프로야구
예측: 13연패,약속의 땅,삼성,포항,삼성 라이온즈,2022,26일

정답: 준우승 김태형,두산 베어스
예측: 준우승,김태형 감독,KT 위즈,준우승,KT,미니클

정답: 강석우,스타 2세,강석우 딸,아빠를 부탁해
예측: 스타석우 딸,강다은,스타 2세

정답: 이브,시청률,방송된 tvN 수목드라마,복귀작,서예지
예측: 서예지 복귀작,시청률 하락세,시청률 반등,서예지,이브

정답: 루친스키,첫 안타 허용,잠실야구장,2021 KBO리그,NC 다이노스,LG 트윈스,경기
예측: 루친스키,첫 안타를 허용,잠실야구장,2021 KBO리그 NC 다이노스,LG 트윈스

정답: 물리치료 버스 운영,의료취약


예측: 경미래교육파주캠퍼스,오는 11월 25일까지,학교 밖 청소년,사회적 배려계층 청소년 20명,찾아가는 영어교육,코로나19,교육 사각지대 해소

정답: 개혁입법,건설안전특별법,민주노총 건설노조,결의대회
예측: 건설노조,총력투쟁 결의대회,행진,건설안전특별법 제정 및 개혁입법 쟁취

정답: 김강률,최용제,두산 베어스,SSG 랜더스,경기,8-5 승리,2021 신한은행 SOL KBO리그
예측: 두강률,최용제,2021 신한은행 SOL KBO리그,두산 베어스

정답: 롯데칠성음료,전년동기,영업이익,순이익
예측: 롯데칠성음료,1분기 영업이익,작년 동기 대비,416.2% 증가,롯데칠성음료,영업이익,롯데칠성음료

정답: 가정법원,윤정희,성년후견,면접조사,알츠하이머병,프랑스 거주,바이올리니스트 백진희
예측: 알법한 면담,성년후견,윤정희,서울가정법원

정답: 국고 지원,학교 예산,줌 유료화,교육부
예측: 교육 유료화,교육부

정답: 일론머스크,테슬라,버니센더스 상원의원,부유세
예측: 샌론 머스크 테슬라 최고경영자,샌더스 상원의원

정답: 화요청백전,모태범,홍현희,쩍벌,굴욕,스피드스케이팅 선수,청백팀
예측: 화요청백전,모태범,홍현희,쩍벌,굴욕 선사

정답: 석유공사,친환경 선도 기업,환골탈태,탄소 감축 노력,코로나 사태,석유 수요 급감,생존을 위한 유연성
예측: 친석유공사,환골탈태,탄소 감축,코로나 사태,핵심,핵심,국제유가,국제유동,탄소

정답: 항저우 亞 게임 유도 사령탑,김미정,여자 유도 대표팀 사령탑
예측: 김미정,황희태,항저우 亞 게임 유도 사령탑,김미정,여자 지도자,대한유도회,항저우아시안게임 여자대표팀

정답: 탕웨이,페르소나,마리끌레르,헤어질결심
예측: 헤질 결심

정답: 밤 초콜릿 테린,요리책,프랑스 쿡북,재료,조리법,한국 포털 사이트,김수연 세미콜론 편집자
예측: 밤콜릿 테린,프랑스 요리책,프랑스 쿡북

정답: 주택화재,완주 카센터
예측: 완 완주 카센터,불씨,불씨,카센터 뒷편 주택

정답: 김맹윤씨,부친상
예측: 김맹윤씨,김맹윤씨,서울시 서대문구 신촌 세브란스병